**Functions4animatedKomplexOscillations**

Diese Funktionen erzeugen aus gegebenen Kreisfrequenzen, Amplituden, Phasen und gegebener Zeitskala komplexe Schwingungen $z_i$, die zu $z_{sum}$ addiert werden, und graphisch als rotierende Vektoren dargestellt werden. Ihre zeitabhängige Projektionen auf die reelle und imaginäre Achse werden ebenfalls gezeigt.  

Kreisfrequenzen, Amplituden, Phasen und Zeitskala müssen jeweils in Form von python-Listen übergeben werden.
Die `male_Schwingungen_z_und_y_von_t`-Funktion braucht noch einen Titel des Graphen als Argument.

In [1]:
def erzeuge_z_Vektoren_und_Summe(W,A,P,T): 
    # Symbole für komplexe Schwingungen definieren: Amplitude a, Phase phi, Kreisfrequenz w, Zeit t
    phi, t=sy.symbols('phi t', real=True) # es sollen reelle Zahlen sein, daher die Option real=True 
    a, w=sy.symbols('a w') 
    
    # Allgemein: Schwingung in komplexer Exponentialform 
    z=a*sy.exp(sy.I*(w*t+phi))   
    
    # einen leeren z_alle Vektor vorbereiten, wo alle komplexen Schwingungen stehen werden, 
    # z_alle[0] für die erste Schwingung, z_alle[1] für zweite, usw. 
    # der Vektor hat erstmal die Länge gleich der Zahl der Summanden, die wir angegeben haben: len(A) oder len(P)
    z_alle=[None]*len(A)
    # Kreisfrquenz, Amplituden, Phasen, Zeitpunkte durch W,A,P,T ersetzen:
    for i in range(len(A)):    # für alle n Komponenten des Vektors z_alle, von z_alle[0] bis z_alle[n-1]
        z_alle[i]=z.subs({w: W[i], a: A[i], phi: P[i]}).evalf() # z_alle[0], erste Schwingung,  hat Kreisfrequenz W[0], Amplitude A[0] und Phase P[0], das Gleiche für die zweite Schwingung z_alle[1], usw.
        z_alle[i]=[z_alle[i].subs({t: tn}).evalf() for tn in T] # Symbol t durch Werte aus dem Zeitbereich T ersetzen, 
                                                            # mittels 'list comprehension' (for tn in T), 
                                                            # wo tn ein Element des T-Vektors ist, das alle Elemente des T's durchläuft
                                                            # einzelne z_alle-Komponenten werden damit zu Listen, 
                                                            # die jeweils die gleiche Länge haben, wie T   

    # Summe der Vektoren z_alle[0]...z_alle[n-1] am Ende des z_alle-Vektors mittels z_alle.append()-numpy-Funktion erstellen, 
    # falls mehr als 1 z_alle Vektor, also Länge des Vektors z_alle, len(z_alle)>1): 
    # Die Summe wird eine numpy-Array:
    if len(z_alle)>1:
        z_alle.append(np.sum(z_alle,axis=0))  
        
    return z_alle

Jetzt können wir so vorbereitete, komplexe Zahlen als Vektoren an unsere Grafik-Funktion übergeben. 

In [2]:
def male_Schwingungen_z_und_y_von_t(z,t,titel):
    
    #Plot definieren
    fig, ax = plt.subplots(2,2,figsize=(2*5,2*5)) # zwei plots nebeneinanader, ein unten, jeweils mit der Größe 5x5 Zoll

    # Quiver plot, ax[0,0], Bewegung in der Gaußschen Ebene, vorbereiten:

    # Anzahl der z-Vektoren ist die Länge des Vektors z:
    N=len(z)
    
    # Anfangsbedingungen: t=0
    time=0
    re_all=[sy.re(z[i][time]) for i in range(N)] # Realteil  der komplexen Zahlen z1, z2,... z_sum
    im_all=[sy.im(z[i][time]) for i in range(N)] # Imaginärteil  der komplexen Zahlen z1, z2,... z_sum

    # origin-Punkt der z-Vektoren
    X, Y = [0,0]
    # End-Punkt der z-Vektoren
    V0=np.array([re_all,im_all]).astype(np.float64)# .astype(np.float64) -> damit sympy-float in numpy-float gut rüber kommt  
    U=V0[0,:] # x-Komponenten (real)  
    V=V0[1,:] # y-Komponenten (imaginär)

    # Anfangs-quiver plot, ax[0] 
    Q = ax[0,0].quiver(X, Y, U, V, angles='xy', scale_units='xy', scale=1, color=['r','b','g', 'm','y','k','r','b','g'])

    # Achsen-Grenzen und Beschriftungen
    if N>1:
        amp=[sy.Abs(z[i][time]) for i in range(N-1)] # Amplituden der einzelnen Komponenten, ohne Summe 
    else:
        amp=[sy.Abs(z[0][time])]
    extr=1.2*float(sum(amp))   # max/min der z,y Achsen wir durch Summe der Amplituden definiert, float(): sympy-float-number->python float
    ax[0,0].set_xlim(-extr,extr)
    ax[0,0].set_ylim(-extr,extr)
    ax[0,0].set_xlabel('Re(z)')         
    ax[0,0].set_ylabel('Im(z)')
    ax[0,0].grid()
    ax[0,0].set_title(titel)

    # Labels
    Farbe=['r','b','g', 'm','y','k','r','b','g']
    qk = [ax[0,0].quiverkey(Q, 0.9, 0.9-.1*i, 3,  r'$z_{'+str(i+1)+'}$', color=Farbe[i],) for i in range(N-1)],
    [ax[0,0].quiverkey(Q, 0.9, 0.9-.1*(N-1), 3,  r'$z_{sum}$', color=Farbe[N-1],)]

    
    # Linien plots: ax[1,0]-Projektion auf die reale Achse und ax[0,1]-Projektion auf die imaginäre Achse

    # Anfangsbedingungen für beide Linienplots: re-Teil ax[1,0] und im-Teil ax[0,1], Werte für Zeit t=0, 
    zeit = [t[time]] # Zeit-Vektor
    x = [[] for i in range(N)] # leere Vektoren für re(z) vorbereiten, in ein y Vektor packen
    [x[i].append(U[i]) for i in range(N)] # Anfangswerte re(z) in diese Vektoren aus Vektor U schreiben
    y = [[] for i in range(N)] # leere Vektoren für im(z) vorbereiten, in ein y Vektor packen
    [y[i].append(V[i]) for i in range(N)] # Anfangswerte in diese Vektoren aus Vektor V schreiben   

    # Linien plot, ax[1,0]: Projektionen auf die reale Achse, Zeit-Achse vertikal, re(z) horizontal, 
    # damit man die Projektion gut sehen kann       

    # Linien initialisieren  
    line1 = [None]*N 
    for i in range(N-1):
        line1[i], = ax[1,0].plot(x[i],zeit, Farbe[i]+'--', label='re($z_{'+str(i+1)+'}$)') # gestrichelte Linien für Summanden
    line1[N-1], = ax[1,0].plot(x[N-1],zeit, Farbe[N-1]+'-', label='re($z_{sum}$)')    # durchgezogene Linie für die Summe  

    # Achsen-Grenzen und Beschriftungen:
    ax[1,0].set_xlim(-extr,extr)
    ax[1,0].set_ylim(t[-1],t[0])    # das letzte und erste Element des tt-Vektors
    ax[1,0].set_ylabel('Zeit')       
    ax[1,0].set_xlabel('Re(z)')     
    ax[1,0].set_title('Realteil der komplexen Schwingungen vs. Zeit')
    ax[1,0].legend()
    ax[1,0].grid()

    # Linien plot, ax[0,1]: Projektionen auf die imaginäre Achse 
    
    # Linien initialisieren  
    line2 = [None]*N 
    for i in range(N-1):
        line2[i], = ax[0,1].plot(zeit, y[i], Farbe[i]+'--', label='im($z_{'+str(i+1)+'}$)') # gestrichelte Linien für Summanden
    line2[N-1], = ax[0,1].plot(zeit, y[N-1], Farbe[N-1]+'-', label='im($z_{sum})$')    # durchgezogene Linie für die Summe  

    # Achsen-Grenzen und Beschriftungen
    ax[0,1].set_ylim(-extr,extr)
    ax[0,1].set_xlim(t[0],t[-1]) # das erste und letzte Element des tt-Vektors
    ax[0,1].set_xlabel('Zeit')         
    ax[0,1].set_ylabel('Im(z)')     
    ax[0,1].set_title('Imaginärteil der komplexen Schwingungen vs. Zeit')
    ax[0,1].legend()
    ax[0,1].grid()
          
    ax[1,1].remove() # das vierte plot brauchen wir nicht
    
    (Q, line1, line2,)=update_quiver_and_line(time, Q, X, Y, z, zeit, x, y, line1, line2, t)
    return Q, X, Y, fig, line1, line2, x, y, zeit  


In der Grafik-Funktion oben benutzen wir eine `update_quiver_and_line`-Funktion, welche  noch definiert werden muss. In dieser Funktion definieren wir, welche Größen wie animiert werden sollen:

In [3]:
def update_quiver_and_line(time, Q, X, Y, z, zeit, x, y, line1, line2, t): 
    """updates the horizontal and vertical vector components by a fixed increment (element time from t-array) on each frame
    """
    re_all=[sy.re(z[i][int(time)]) for i in range(len(z))] # Realteil  der komplexen Zahlen z1, z2, z_sum
    im_all=[sy.im(z[i][int(time)]) for i in range(len(z))] # Imaginärteil  der komplexen Zahlen z1, z2, z_sum

    # End-Punkt der z-Vektoren für den Zeitpunkt 'time' auf dem Plot Q (ax[0]) aktualisieren
    V0=np.array([re_all,im_all]).astype(np.float64) # .astype(np.float64) braucht man, da sonst Probleme mit Konvertierung von sympy in numpy auftreten
    U=V0[0,:] # x-Komponenten (real)  
    V=V0[1,:] # y-Komponenten (imaginär)
    Q.set_UVC(U,V)
    
    # Linienplots (ax[1,0] und ax[0,1]) zum Zeitpunkt 'time' aktualisieren
    zeit.append(t[time])                                   # Zeit-Vektor zeit um neuen Zeitpunkt 'time' verlängern
    [x[i].append(U[i]) for i in range(len(z))]             # x-Vektor um Vektor U verlängern
    [line1[i].set_data(x[i],zeit) for i in range(len(z))]  # re(z)-Linien aktualisieren
    [y[i].append(V[i]) for i in range(len(z))]             # y-Vektor um Vektor V verlängern
    [line2[i].set_data(zeit, y[i]) for i in range(len(z))] # im(z)-Linien aktualisieren
    return  Q, line1, line2

Anwendungsbeispiel in einem externen Notebook
<!--
%matplotlib notebook 
import matplotlib.pyplot as plt 
from matplotlib import animation  # für Animationen
import numpy as np
import sympy as sy

%run Functions4animatedKomplexOscillations.ipynb

omega=[2*2*sy.pi,2*2*sy.pi]      # Kreisfrequenz, gleich für beide Schwingungen
amp  = [10, 5]                   # Amplitude der ersten, zweiten, etc. Schwingung
phase= [0, sy.pi/2]              # Phase der ersten, zweiten, etc. Schwingung
T=np.linspace(0,2,100)           # Zeitbereich tt zum Zeichnen definieren
Titel='2 z-Vektoren und ihre Summe in Gauß-Ebene' 

z_all=erzeuge_z_Vektoren_und_Summe(omega, amp, phase, T)
Q, X, Y, fig01, line1, line2, x, y, zeit = male_Schwingungen_z_und_y_von_t(z_all,T,Titel)
Intervall=80 # in [ms]: wie schnell soll die Grafik aktualisiert werden
anim = animation.FuncAnimation(fig01, update_quiver_and_line, fargs=(Q, X, Y, z_all, zeit, 
                                                                     x, y, line1, line2, T),
                               interval=Intervall, blit=True)
-->